In [ ]:
# DIRECTORY SET
import os
import sys
import traceback
from pathlib import Path
base_dir=Path(os.getcwd()).parent.parent
os.chdir(base_dir)
print(os.getcwd())

# ENVIRONMENT VARIABLES
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

# Enable autoreload
%load_ext autoreload
%autoreload 2

# Import custom modules
from destinyapp.models import StreamRecapData

from core import services
from core import utils
from core import controller

In [ ]:
# Input parameter
video_id="OqVH_MTBQ6k"

# Discord Message Testing

In [ ]:
discord_recaps_to_send=await services.discord.DiscordMessageHandler.compile_discord_messages([video_id])

In [ ]:
await services.discord.DiscordMessageHandler.send_discord_recaps(discord_recaps_to_send)

In [ ]:
async def generate_plot(video_id):
    stream_recap_data=await utils.get_recap_data(video_id)

    text_chunks_no_overlap = await create_text_chunks(stream_recap_data.transcript, 0)

    chunk_batches = await generate_text_chunk_batches(text_chunks_no_overlap)

    topic_annotations_str = await annotate_major_minor_topics(stream_recap_data.recap)

    major_topics, minor_topics = process_topic_annotations_str(topic_annotations_str)

    responses, annotated_results=await annotate_all_batches(chunk_batches, topic_annotations_str)
    
    segments, category_locations, color_dict = await create_segments(stream_recap_data.linked_transcript, annotated_results, major_topics, stream_recap_data.transcript)

    clickable_areas=await create_and_save_plot(video_id, segments, category_locations, color_dict)

    clickable_areas, base64_plot_image = clickable_and_plot_image_finalization(video_id, clickable_areas)

    return base64_plot_image, clickable_areas, annotated_results

# Plot data generation

### Get data

In [ ]:
stream_recap_data=await utils.get_recap_data(video_id)

### Make raw data chunks to process

In [ ]:
text_chunks_no_overlap = await services.RV.create_text_chunks(stream_recap_data.transcript, 0)

### Batch chunks together so the llm can process them in chunks

In [ ]:
chunk_batches = await services.RV.generate_text_chunk_batches(text_chunks_no_overlap)

### Get the topic annotations for the recap

In [ ]:
topic_annotations_str = await services.RV.annotate_major_minor_topics(stream_recap_data.recap)
major_topics, minor_topics = services.RV.process_topic_annotations_str(topic_annotations_str)

#### Annotate the batches according to the topic annotations gathered

In [ ]:
responses, annotated_results=await services.RV.annotate_all_batches(chunk_batches, topic_annotations_str)

In [ ]:
# save annotated results to json
import json
data_to_save={"annotated_results":annotated_results, "major_topics":major_topics, "minor_topics":minor_topics}
with open(f"testing_visualization_data_gen.json", "w") as f:
    json.dump(data_to_save, f)

# Process generated data

### Put the annotated results into a data structure that will eventually be saved

In [ ]:
# load data from testing_visualization_annotation_results.json
import json
with open(f"testing_visualization_data_gen.json", "r") as f:
    data_gen=json.load(f)

annotated_results=data_gen["annotated_results"]
major_topics=data_gen["major_topics"]
minor_topics=data_gen["minor_topics"]

stream_recap_data=await utils.get_recap_data(video_id)

In [ ]:
plot_segments, category_locations, color_dict = await services.RV.create_segments(stream_recap_data.linked_transcript, annotated_results, major_topics, stream_recap_data.transcript)

# PLOT

### Initialize Plot

dictionary of PlotCategoryObject s
PlotCategoryObject:
    segments
    abstractions

In [ ]:
plot_obj=services.RV.PlotObject()

In [ ]:
plot_obj.segments[0]

In [ ]:
test_obj.Segment("test",1,2,3)

In [ ]:
test_obj.segments.append(test_obj.Segment("test",1,2,3))

In [ ]:
test_obj.segments[0].

In [ ]:
# Create the plot with a specific gray background
background_color=96
fig, ax = plt.subplots(figsize=(12, 10))
hex_background_color = '#%02x%02x%02x' % (int(background_color), int(background_color), int(background_color))
fig.patch.set_facecolor(hex_background_color)  # Set figure background to [96, 96, 96]
ax.set_facecolor(hex_background_color)  # Set axes background to [96, 96, 96]
target_plot_width=10

In [ ]:
for i, segment in enumerate(segments):
    ax.add_patch(plt.Rectangle((current_x, 0), segment['width'], bar_height, 
                            facecolor=segment['color'], edgecolor='white'))
    category_info[segment['category']]['total_width'] += segment['width']
    category_info[segment['category']]['segments'].append((current_x, segment['width']))
    category_info[segment['category']]['color'] = segment['color']
    
    # Add clickable areas
    # this but a float and not a string f"{int((current_x+clickable_area_x_offset)*10)},{int((10-bar_height-clickable_area_y_offset)*10)},{int((current_x+segment['width']-clickable_area_x_offset)*10)},{int(10*(10-clickable_area_y_offset))}"}

    coord_list=[(current_x)*10, (10-(bar_height*clickable_y_height_multiplier)-clickable_area_y_offset)*10, (current_x+segment['width'])*10, 10*(10-clickable_area_y_offset)]

    # clickable_area={"alt":href, "title":href, "href":href, "coords": f"{int((current_x+clickable_area_x_offset)*10)},{int((10-bar_height-clickable_area_y_offset)*10)},{int((current_x+segment['width']-clickable_area_x_offset)*10)},{int(10*(10-clickable_area_y_offset))}"}
    href=href_base+str(int(segment["start"]))

    clickable_area={"alt":href, "title":href, "href":href, "coords": coord_list}

    clickable_areas.append(clickable_area)

    current_x += segment['width']

total_width = current_x

In [ ]:
async def create_and_save_plot(video_id, segments, category_locations, color_dict):
    # Create the plot with a specific gray background
    fig, ax = plt.subplots(figsize=(12, 10))

    # convert [background_color, background_color, background_color] to hex value
    hex_background_color = '#%02x%02x%02x' % (int(background_color), int(background_color), int(background_color))

    fig.patch.set_facecolor(hex_background_color)  # Set figure background to [96, 96, 96]
    ax.set_facecolor(hex_background_color)  # Set axes background to [96, 96, 96]
    target_plot_width=10


    # Plot BAR 
    current_x = 0
    category_info = defaultdict(lambda: {"total_width": 0, "segments": []})

    bar_height = 1.5
    bar_height = 1.0

    clickable_area_x_offset=0.085
    clickable_area_y_offset=0.1
    clickable_y_height_multiplier=1.22
    clickable_areas=[]
    href_base="https://youtu.be/"+video_id+"?t="


    for i, segment in enumerate(segments):
        ax.add_patch(plt.Rectangle((current_x, 0), segment['width'], bar_height, 
                                facecolor=segment['color'], edgecolor='white'))
        category_info[segment['category']]['total_width'] += segment['width']
        category_info[segment['category']]['segments'].append((current_x, segment['width']))
        category_info[segment['category']]['color'] = segment['color']
        
        # Add clickable areas
        # this but a float and not a string f"{int((current_x+clickable_area_x_offset)*10)},{int((10-bar_height-clickable_area_y_offset)*10)},{int((current_x+segment['width']-clickable_area_x_offset)*10)},{int(10*(10-clickable_area_y_offset))}"}

        coord_list=[(current_x)*10, (10-(bar_height*clickable_y_height_multiplier)-clickable_area_y_offset)*10, (current_x+segment['width'])*10, 10*(10-clickable_area_y_offset)]

        # clickable_area={"alt":href, "title":href, "href":href, "coords": f"{int((current_x+clickable_area_x_offset)*10)},{int((10-bar_height-clickable_area_y_offset)*10)},{int((current_x+segment['width']-clickable_area_x_offset)*10)},{int(10*(10-clickable_area_y_offset))}"}
        href=href_base+str(int(segment["start"]))

        clickable_area={"alt":href, "title":href, "href":href, "coords": coord_list}

        clickable_areas.append(clickable_area)

        current_x += segment['width']

    total_width = current_x
    print("Total width:", total_width)







    # sort category items by the same order as the category_locations
    category_info=dict(sorted(category_info.items(), key=lambda item: category_locations[item[0]]))


    # Calculate Circle Padding
    circle_zone_size=9
    circle_y = 3.5
    circle_y = 2.78
    circle_size_variable = 0.15
    circle_size_variable = 0.13
    circle_base_size_variable=0.3
    circle_base_size_variable=0.41

    current_x=(target_plot_width-circle_zone_size)/2
    circle_x_locations={}
    total_circles_width=0
    def get_circle_width(total_width):
        return (((np.sqrt(total_width) * circle_size_variable)*2)+circle_base_size_variable)

    for category, info in category_info.items():
        if category == 'non categorized':
            pass
        else:
            circle_width=get_circle_width(info['total_width'])
            circle_x_locations[category]=circle_width
            total_circles_width+=circle_width
    number_of_circles=len(circle_x_locations)
    between_circle_padding=(circle_zone_size-total_circles_width)/(number_of_circles+1.15)



    alterating_bool=False
    vertical_offset_value=0.5
    vertical_offset_value=0.65
    circle_centers = []
    for category, info in category_info.items():
        if category == 'non categorized':
            continue

        circle_applied_size=get_circle_width(info['total_width'])

        # Calculate x position for the circle (center of all segments of this category)
        circle_x = current_x + circle_applied_size
        if alterating_bool:
            alterating_bool=False
            vertical_offset=-vertical_offset_value
        else:
            vertical_offset=vertical_offset_value
            alterating_bool=True

        # Store circle center, color, and size for later use
        circle_size = np.sqrt(info['total_width']) * circle_size_variable
        circle_applied_y=circle_y+vertical_offset
        circle_centers.append((circle_x, circle_applied_y, info['color'], circle_size))
        
        # DRAW LINES
        for segment_start, segment_width in info['segments']:
            segment_center = segment_start + segment_width / 2
            ax.plot([segment_center, circle_x], [bar_height, circle_applied_y], 
                    color=info['color'], linewidth=1)  
            
        # PLOT CIRCLES
        circle = plt.Circle((circle_x, circle_applied_y), circle_applied_size, 
                            facecolor=info["color"], edgecolor='white', zorder=10)
        ax.add_artist(circle)

        # PLOT CIRCLE LABELS, set width to circle size and wrap
        text_wrap=textwrap.fill(category, width=12)
        bubble_font_size_text=14
        if info["color"]=="yellow":
            # make it bold and have a white border
            ax.text(circle_x, circle_applied_y, text_wrap, ha='center', va='center', color='black', fontsize=bubble_font_size_text, zorder=11, fontweight='bold')#, bbox=dict(facecolor='yellow', edgecolor='black', boxstyle='round,pad=0.2'))
        else:
            ax.text(circle_x, circle_applied_y, text_wrap, ha='center', va='center', color='white', fontsize=bubble_font_size_text, zorder=11, fontweight='bold')

        # current_x += (info['total_width']*circle_mutlipler)+0.1
        current_x +=between_circle_padding+circle_applied_size
        print("Circle current x:", current_x)



    # Add central white circle
    central_y = 6
    central_y = 5
    central_circle = plt.Circle((total_width/2, central_y), 0.25, 
                                facecolor='white', edgecolor='black', zorder=12)
    ax.add_artist(central_circle)

    # Connect category circles to central circle with colored lines
    for circle_x, circle_y, color, circle_size in circle_centers:
        ax.plot([circle_x, total_width/2], [circle_y, central_y], 
                color=color, linewidth=3, linestyle='-', alpha=0.7, zorder=9)  # Increased linewidth

    # Add white line extending upward from central circle
    top_y = 100  # Adjust this value to change the length of the line
    ax.plot([total_width/2, total_width/2], [central_y, top_y], 
            color='white', linewidth=3, solid_capstyle='round')  # Added white line

    # Customize the plot
    ax.set_xlim(0, total_width)
    ax.set_ylim(0, 8)
    ax.set_aspect('equal', adjustable='box')
    ax.axis('off')

    # # Add a Legend
    # legend_handles = [plt.Rectangle((0, 0), 1, 1, color=color) for color in color_dict.values()]
    # legend_labels = list(color_dict.keys())
    # ax.legend(legend_handles, legend_labels, loc='upper left', frameon=False)
    # # increase legend font size
    # plt.setp(ax.get_legend().get_texts(), fontsize='21')
    # plt.tight_layout()

    # save plot
    global images_folder
    if not os.path.isdir(images_folder):
        os.makedirs(images_folder)
    plt.savefig(images_folder+video_id+'_plot.png', dpi=300, bbox_inches='tight')

    return clickable_areas